In [ ]:
import os
import pandas as pd

# Define the base directory
base_dir = './data/classification'

# Initialize an empty list to store the data
data = []

# Walk through the directory structure
for root, dirs, files in os.walk(base_dir):
    # Initialize variables to store scores
    test_score = None
    training_score = None
    
    for file in files:
        if file in ['test_scores', 'training_scores']:
            # Extract variable values from the directory structure
            parts = root.split('/')
            classification_id = parts[-6]
            encoding = parts[-5]
            ansatz = parts[-4]
            split = int(parts[-3][5:])  # Extract integer value following 'split'
            num_ansatz_layers = int(parts[-2][1])  # Extract integer following 'a'
            datapoint = int(parts[-1][1:])  # Extract integer value from p{1}

            # Read the score from the file
            with open(os.path.join(root, file), 'r') as f:
                score = float(f.read().strip())

            # Assign score to the appropriate variable
            if file == 'test_scores':
                test_score = score
            elif file == 'training_scores':
                training_score = score
    
    # Append the data to the list if both scores are found
    if test_score is not None and training_score is not None:
        data.append({
            'LSFJOBID': classification_id,
            'encoding': encoding,
            'ansatz': ansatz,
            'split': split,
            'num_ansatz_layers': num_ansatz_layers,
            'datapoint': datapoint,
            'training_score': training_score,
            'test_score': test_score
        })

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Sort the DataFrame by all columns except training and test scores
df.sort_values(by=['LSFJOBID', 'encoding', 'ansatz', 'split', 'num_ansatz_layers', 'datapoint'], inplace=True)



# Save the DataFrame to an Excel file (append mode)
excel_file = f"{base_dir}/scores.xlsx"
if os.path.exists(excel_file):
    existing_df = pd.read_excel(excel_file, engine='openpyxl')
    df = pd.concat([existing_df, df], ignore_index=True)

df.to_excel(excel_file, index=False)